In [41]:
#https://towardsdatascience.com/learn-word2vec-by-implementing-it-in-tensorflow-45641adaf2ac

import numpy as np
import tensorflow as tf
corpus_raw = 'He is the king . The king is royal . She is the royal  queen '
# convert to lower case
corpus_raw = corpus_raw.lower()

In [13]:
corpus_raw

'he is the king . the king is royal . she is the royal  queen '

In [14]:
words = []
for word in corpus_raw.split():
    if word != '.': # because we don't want to treat . as a word
        words.append(word)
words = set(words) # so that all duplicate words are removed
word2int = {}
int2word = {}
vocab_size = len(words) # gives the total number of unique words
for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [16]:
print(word2int['queen'])
print(int2word[3])

3
queen


In [18]:
# raw sentences is a list of sentences.
raw_sentences = corpus_raw.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())
    
print(sentences)

[['he', 'is', 'the', 'king'], ['the', 'king', 'is', 'royal'], ['she', 'is', 'the', 'royal', 'queen']]


In [31]:
data = []
WINDOW_SIZE = 2
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

In [32]:
data

[['he', 'is'],
 ['he', 'the'],
 ['is', 'he'],
 ['is', 'the'],
 ['is', 'king'],
 ['the', 'he'],
 ['the', 'is'],
 ['the', 'king'],
 ['king', 'is'],
 ['king', 'the'],
 ['the', 'king'],
 ['the', 'is'],
 ['king', 'the'],
 ['king', 'is'],
 ['king', 'royal'],
 ['is', 'the'],
 ['is', 'king'],
 ['is', 'royal'],
 ['royal', 'king'],
 ['royal', 'is'],
 ['she', 'is'],
 ['she', 'the'],
 ['is', 'she'],
 ['is', 'the'],
 ['is', 'royal'],
 ['the', 'she'],
 ['the', 'is'],
 ['the', 'royal'],
 ['the', 'queen'],
 ['royal', 'is'],
 ['royal', 'the'],
 ['royal', 'queen'],
 ['queen', 'the'],
 ['queen', 'royal']]

In [33]:
# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp
x_train = [] # input word
y_train = [] # output word
for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))
# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [38]:
print(x_train.shape)
print(y_train.shape)
print(x_train)

(34, 7)
(34, 7)
[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [39]:
# making placeholders for x_train and y_train
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

In [40]:
EMBEDDING_DIM = 5 # you can choose your own number
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_representation = tf.add(tf.matmul(x,W1), b1)

In [42]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

In [43]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) #make sure you do this!
# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

loss is :  3.256593
loss is :  3.0084
loss is :  2.8142033
loss is :  2.659947
loss is :  2.5356796
loss is :  2.4342604
loss is :  2.3504689
loss is :  2.2804556
loss is :  2.22138
loss is :  2.1711252
loss is :  2.1280856
loss is :  2.0910046
loss is :  2.0588655
loss is :  2.0308247
loss is :  2.006174
loss is :  1.9843165
loss is :  1.9647547
loss is :  1.9470792
loss is :  1.930956
loss is :  1.9161171
loss is :  1.9023489
loss is :  1.8894824
loss is :  1.8773832
loss is :  1.8659464
loss is :  1.8550869
loss is :  1.8447381
loss is :  1.8348447
loss is :  1.8253626
loss is :  1.8162543
loss is :  1.8074889
loss is :  1.7990401
loss is :  1.7908852
loss is :  1.7830043
loss is :  1.7753805
loss is :  1.7679985
loss is :  1.7608443
loss is :  1.7539065
loss is :  1.7471732
loss is :  1.7406348
loss is :  1.7342824
loss is :  1.7281073
loss is :  1.7221019
loss is :  1.7162591
loss is :  1.7105722
loss is :  1.7050351
loss is :  1.6996418
loss is :  1.6943872
loss is :  1.6892658
l

loss is :  1.3551201
loss is :  1.3550119
loss is :  1.3549047
loss is :  1.3547975
loss is :  1.354691
loss is :  1.3545849
loss is :  1.3544792
loss is :  1.3543739
loss is :  1.3542694
loss is :  1.3541651
loss is :  1.3540611
loss is :  1.3539578
loss is :  1.353855
loss is :  1.3537526
loss is :  1.3536505
loss is :  1.353549
loss is :  1.3534478
loss is :  1.3533471
loss is :  1.3532468
loss is :  1.3531471
loss is :  1.3530476
loss is :  1.3529487
loss is :  1.3528502
loss is :  1.3527521
loss is :  1.3526545
loss is :  1.3525572
loss is :  1.3524604
loss is :  1.3523641
loss is :  1.352268
loss is :  1.3521725
loss is :  1.3520772
loss is :  1.3519826
loss is :  1.3518882
loss is :  1.3517942
loss is :  1.3517008
loss is :  1.3516076
loss is :  1.3515149
loss is :  1.3514224
loss is :  1.3513306
loss is :  1.3512391
loss is :  1.3511478
loss is :  1.351057
loss is :  1.3509667
loss is :  1.3508768
loss is :  1.3507872
loss is :  1.3506978
loss is :  1.3506091
loss is :  1.35052

loss is :  1.3333528
loss is :  1.3333296
loss is :  1.3333061
loss is :  1.333283
loss is :  1.3332596
loss is :  1.3332365
loss is :  1.3332133
loss is :  1.3331902
loss is :  1.3331674
loss is :  1.3331447
loss is :  1.3331217
loss is :  1.333099
loss is :  1.3330762
loss is :  1.3330537
loss is :  1.3330313
loss is :  1.3330089
loss is :  1.3329864
loss is :  1.3329642
loss is :  1.3329419
loss is :  1.3329196
loss is :  1.3328975
loss is :  1.3328754
loss is :  1.3328536
loss is :  1.3328315
loss is :  1.3328098
loss is :  1.3327878
loss is :  1.3327662
loss is :  1.3327445
loss is :  1.3327228
loss is :  1.3327013
loss is :  1.3326797
loss is :  1.3326583
loss is :  1.3326371
loss is :  1.3326157
loss is :  1.3325945
loss is :  1.3325734
loss is :  1.3325523
loss is :  1.3325312
loss is :  1.3325101
loss is :  1.3324893
loss is :  1.3324684
loss is :  1.3324476
loss is :  1.3324269
loss is :  1.332406
loss is :  1.3323855
loss is :  1.3323649
loss is :  1.3323445
loss is :  1.332

loss is :  1.3271812
loss is :  1.3271736
loss is :  1.3271657
loss is :  1.3271581
loss is :  1.3271505
loss is :  1.3271428
loss is :  1.3271352
loss is :  1.3271276
loss is :  1.3271201
loss is :  1.3271123
loss is :  1.3271049
loss is :  1.3270973
loss is :  1.3270898
loss is :  1.3270822
loss is :  1.3270746
loss is :  1.3270673
loss is :  1.3270597
loss is :  1.3270522
loss is :  1.3270448
loss is :  1.3270373
loss is :  1.32703
loss is :  1.3270227
loss is :  1.3270152
loss is :  1.3270078
loss is :  1.3270005
loss is :  1.3269931
loss is :  1.3269856
loss is :  1.3269783
loss is :  1.326971
loss is :  1.3269638
loss is :  1.3269566
loss is :  1.3269494
loss is :  1.3269421
loss is :  1.3269348
loss is :  1.3269275
loss is :  1.3269204
loss is :  1.3269131
loss is :  1.326906
loss is :  1.3268988
loss is :  1.3268918
loss is :  1.3268846
loss is :  1.3268775
loss is :  1.3268703
loss is :  1.3268633
loss is :  1.3268561
loss is :  1.3268491
loss is :  1.326842
loss is :  1.32683

loss is :  1.3246715
loss is :  1.324668
loss is :  1.3246646
loss is :  1.3246614
loss is :  1.3246579
loss is :  1.3246546
loss is :  1.3246511
loss is :  1.3246478
loss is :  1.3246444
loss is :  1.3246411
loss is :  1.3246378
loss is :  1.3246344
loss is :  1.3246311
loss is :  1.3246279
loss is :  1.3246243
loss is :  1.3246211
loss is :  1.3246177
loss is :  1.3246144
loss is :  1.3246111
loss is :  1.3246077
loss is :  1.3246044
loss is :  1.3246012
loss is :  1.3245978
loss is :  1.3245946
loss is :  1.3245914
loss is :  1.324588
loss is :  1.3245847
loss is :  1.3245815
loss is :  1.3245783
loss is :  1.3245748
loss is :  1.3245717
loss is :  1.3245685
loss is :  1.324565
loss is :  1.324562
loss is :  1.3245587
loss is :  1.3245554
loss is :  1.3245522
loss is :  1.3245491
loss is :  1.3245459
loss is :  1.3245425
loss is :  1.3245394
loss is :  1.3245362
loss is :  1.3245329
loss is :  1.3245298
loss is :  1.3245267
loss is :  1.3245232
loss is :  1.3245202
loss is :  1.3245

...

loss is :  1.3210402
loss is :  1.32104
loss is :  1.3210399
loss is :  1.3210398
loss is :  1.3210397
loss is :  1.3210397
loss is :  1.3210396
loss is :  1.3210393
loss is :  1.3210393
loss is :  1.3210391
loss is :  1.3210391
loss is :  1.321039
loss is :  1.3210388
loss is :  1.3210387
loss is :  1.3210385
loss is :  1.3210385
loss is :  1.3210384
loss is :  1.3210384
loss is :  1.3210382
loss is :  1.3210382
loss is :  1.3210381
loss is :  1.321038
loss is :  1.3210378
loss is :  1.3210378
loss is :  1.3210377
loss is :  1.3210375
loss is :  1.3210375
loss is :  1.3210373
loss is :  1.3210373
loss is :  1.321037
loss is :  1.3210369
loss is :  1.3210369
loss is :  1.3210367
loss is :  1.3210367
loss is :  1.3210366
loss is :  1.3210365
loss is :  1.3210365
loss is :  1.3210363
loss is :  1.3210361
loss is :  1.3210362
loss is :  1.321036
loss is :  1.321036
loss is :  1.3210359
loss is :  1.3210357
loss is :  1.3210356
loss is :  1.3210354
loss is :  1.3210354
loss is :  1.3210351

loss is :  1.3209982
loss is :  1.3209983
loss is :  1.3209982
loss is :  1.3209982
loss is :  1.3209982
loss is :  1.3209981
loss is :  1.3209981
loss is :  1.3209977
loss is :  1.3209977
loss is :  1.3209976
loss is :  1.3209976
loss is :  1.3209975
loss is :  1.3209972
loss is :  1.3209972
loss is :  1.3209971
loss is :  1.3209971
loss is :  1.3209969
loss is :  1.3209969
loss is :  1.3209966
loss is :  1.3209966
loss is :  1.3209965
loss is :  1.3209965
loss is :  1.3209964
loss is :  1.3209964
loss is :  1.3209964
loss is :  1.3209962
loss is :  1.320996
loss is :  1.3209959
loss is :  1.3209959
loss is :  1.3209959
loss is :  1.3209957
loss is :  1.3209957
loss is :  1.3209957
loss is :  1.3209956
loss is :  1.3209953
loss is :  1.3209952
loss is :  1.3209951
loss is :  1.3209951
loss is :  1.3209951
loss is :  1.3209949
loss is :  1.3209949
loss is :  1.3209947
loss is :  1.3209946
loss is :  1.3209945
loss is :  1.3209944
loss is :  1.3209944
loss is :  1.3209943
loss is :  1.3

loss is :  1.3209562
loss is :  1.3209562
loss is :  1.3209562
loss is :  1.3209561
loss is :  1.320956
loss is :  1.3209559
loss is :  1.3209559
loss is :  1.3209558
loss is :  1.3209555
loss is :  1.3209555
loss is :  1.3209554
loss is :  1.3209554
loss is :  1.3209553
loss is :  1.3209553
loss is :  1.3209552
loss is :  1.320955
loss is :  1.3209549
loss is :  1.3209548
loss is :  1.3209548
loss is :  1.3209547
loss is :  1.3209548
loss is :  1.3209546
loss is :  1.3209546
loss is :  1.3209543
loss is :  1.3209543
loss is :  1.3209543
loss is :  1.3209542
loss is :  1.3209542
loss is :  1.3209541
loss is :  1.320954
loss is :  1.320954
loss is :  1.3209538
loss is :  1.3209538
loss is :  1.3209537
loss is :  1.3209536
loss is :  1.3209536
loss is :  1.3209534
loss is :  1.3209532
loss is :  1.3209532
loss is :  1.3209531
loss is :  1.320953
loss is :  1.320953
loss is :  1.320953
loss is :  1.320953
loss is :  1.3209529
loss is :  1.3209527
loss is :  1.3209525
loss is :  1.3209524


loss is :  1.3209203
loss is :  1.3209203
loss is :  1.3209203
loss is :  1.3209201
loss is :  1.32092
loss is :  1.32092
loss is :  1.32092
loss is :  1.32092
loss is :  1.3209199
loss is :  1.3209199
loss is :  1.3209198
loss is :  1.3209196
loss is :  1.3209196
loss is :  1.3209196
loss is :  1.3209194
loss is :  1.3209193
loss is :  1.3209193
loss is :  1.3209193
loss is :  1.3209193
loss is :  1.320919
loss is :  1.320919
loss is :  1.320919
loss is :  1.3209189
loss is :  1.3209188
loss is :  1.3209188
loss is :  1.3209187
loss is :  1.3209186
loss is :  1.3209184
loss is :  1.3209184
loss is :  1.3209183
loss is :  1.3209182
loss is :  1.3209182
loss is :  1.3209182
loss is :  1.3209181
loss is :  1.3209181
loss is :  1.3209181
loss is :  1.3209181
loss is :  1.320918
loss is :  1.3209177
loss is :  1.3209177
loss is :  1.3209176
loss is :  1.3209175
loss is :  1.3209175
loss is :  1.3209175
loss is :  1.3209174
loss is :  1.3209172
loss is :  1.3209172
loss is :  1.3209172
loss

loss is :  1.3208867
loss is :  1.3208867
loss is :  1.3208866
loss is :  1.3208864
loss is :  1.3208864
loss is :  1.3208864
loss is :  1.3208864
loss is :  1.3208864
loss is :  1.3208861
loss is :  1.3208861
loss is :  1.3208861
loss is :  1.320886
loss is :  1.320886
loss is :  1.3208859
loss is :  1.3208858
loss is :  1.3208858
loss is :  1.3208858
loss is :  1.3208857
loss is :  1.3208857
loss is :  1.3208857
loss is :  1.3208855
loss is :  1.3208854
loss is :  1.3208854
loss is :  1.3208854
loss is :  1.3208853
loss is :  1.3208852
loss is :  1.3208852
loss is :  1.3208852
loss is :  1.320885
loss is :  1.320885
loss is :  1.3208848
loss is :  1.3208848
loss is :  1.3208848
loss is :  1.3208847
loss is :  1.3208847
loss is :  1.3208846
loss is :  1.3208846
loss is :  1.3208845
loss is :  1.3208843
loss is :  1.3208843
loss is :  1.3208842
loss is :  1.3208842
loss is :  1.320884
loss is :  1.3208841
loss is :  1.320884
loss is :  1.3208839
loss is :  1.320884
loss is :  1.3208839

In [44]:
print(sess.run(W1))
print('----------')
print(sess.run(b1))
print('----------')

[[-0.15334266 -0.47788453  1.4116399   1.1273073   1.7270285 ]
 [ 1.27613     0.7677011  -1.4470853  -2.248972    1.7693268 ]
 [-0.62983775  0.0282855  -1.8940669   0.7121755  -0.38023645]
 [-0.05781033 -2.1300273  -2.4958942  -0.13673805 -0.58850616]
 [ 0.16076523  0.1536028  -0.65289193  1.7641594   0.2508547 ]
 [-2.1240964  -0.22530286  0.7557802  -1.171494   -1.0600855 ]
 [-1.0690352  -0.7491811  -1.1518878   1.9544979   0.96830904]]
----------
[-1.811901   -0.5854732  -0.7331353   0.47802174 -0.24650033]
----------


In [45]:
vectors = sess.run(W1 + b1)

# if you work it out, you will see that it has the same effect as running the node hidden representation
print(vectors)

[[-1.9652436  -1.0633577   0.67850465  1.605329    1.4805281 ]
 [-0.535771    0.18222791 -2.1802206  -1.7709502   1.5228264 ]
 [-2.4417386  -0.5571877  -2.6272023   1.1901972  -0.62673676]
 [-1.8697113  -2.7155004  -3.2290294   0.34128368 -0.8350065 ]
 [-1.6511357  -0.4318704  -1.3860272   2.2421813   0.00435437]
 [-3.9359975  -0.81077605  0.02264494 -0.69347227 -1.3065859 ]
 [-2.8809361  -1.3346543  -1.8850231   2.4325197   0.72180873]]
